In [1]:
#!/usr/bin/env python3

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [5]:
class Data():
    def __init__(self, train_csv, train_target_csv, cat_features, num_features, target):
        self.cat_features = cat_features
        self.num_features = num_features
        self.target_label = target
        self.data_frame = self.read_data(train_csv, train_target_csv)
        self.target_data = None
        self.Xtrain = None
        self.ytrain = None
        self.Xtest = None
        self.ytest = None
        self.pre_process()
        self.split_data()

    def convert_to_cat_dtype(self):
        for feat in self.cat_features:
            self.data_frame[feat] = self.data_frame[feat].astype("category")
    
    def scale_numeric_features(self):
        num_data = self.data_frame[self.num_features].values
        std_scaler = StandardScaler()
        scaled_values = std_scaler.fit_transform(num_data)
        num_df = pd.DataFrame(scaled_values, columns=self.num_features)
        test= pd.concat([num_df, self.data_frame[self.cat_features]], axis=1)
        
    def encode_cat_features(self):
        new_df = self.data_frame[self.num_features].copy()
        for cat_feature in self.cat_features:
            encoded_output = pd.get_dummies(self.data_frame[cat_feature])
            new_df = pd.concat([new_df, encoded_output], axis=1)
        self.data_frame = new_df
        
    def get_target(self):
        return self.data_frame.loc["salary"].copy()
            
    def clean_data(self, df):
        total_df = df.copy()
        total_df.drop_duplicates(inplace=True)
        clean_df = total_df[total_df.salary >0].copy()
        return clean_df

    def pre_process(self):
        self.convert_to_cat_dtype()
        self.scale_numeric_features()
        self.encode_cat_features()
        
    def read_data(self, csv1, csv2):
        df = pd.read_csv(csv1)
        df2 = pd.read_csv(csv2)
        total_df = pd.concat([df, df2],axis=1)
        clean_df = self.clean_data(total_df)
        self.target_data = clean_df["salary"].copy()
        return clean_df
    
    def split_data(self):
        consolidated_df = pd.concat([self.data_frame, self.target_data],axis=1)
        train, test = train_test_split(consolidated_df, test_size=0.2)
        self.Xtrain = train.iloc[:, :-1]
        self.ytrain = train.iloc[:,-1]
        self.Xtest = test.iloc[:, :-1]
        self.ytest = test.iloc[:,-1]


class Models:
    def __init__(self, data):
        self.Xtrain = data.Xtrain
        self.ytrain = data.ytrain
        self.Xtest = data.Xtest
        self.ytest = data.ytest
        self.models = []
        self.mse = {}
        self.cross_val_scores = {}
        self.predictions = None
        self.best_model = None
        
    
    def add_model(self, model):
        self.models.append(model)
        
    
    def cross_validate(self, k):
        for model in self.models:
            model.fit(self.Xtrain, self.ytrain)
            cross_evaluation = cross_val_score(model, self.Xtrain, self.ytrain, cv=k, scoring="neg_mean_squared_error")
            score = np.mean(np.negative(cross_evaluation))
            self.model_info[model] = score
        
    def select_best_model(self):
        self.best_model = min(self.model_info, key=self.model_info.get)
        
    def summarize(self):
        for model in self.models:
            print(model + "\n")
            print(self.model_info[model] + "\n")
        print(self.best_model)
            
    def hyper_param_tune(self, model, params):
        grid_search = GridSearchCV(model, params, cv=5, scoring = "neg_mean_squared_error")
        grid_search.fit(self.Xtrain, self.ytrain)
        return grid_search
        
            
    def run(self):
        self.cross_validate(5)
        self.select_best_model()
        self.summarize()
        
            
    
        
            

In [6]:
dataset_features = "data/train_features.csv"
dataset_target = "data/train_salaries.csv"
cat_features = ["companyId", "jobType", "degree", "major", "industry"]
num_features = ["milesFromMetropolis", "yearsExperience"]

lin_alg = LinearRegression()
rfr = RandomForestRegressor()
gbr = GradientBoostingRegressor()

param_grid = [{'n_estimators':[3,10,30], 'max_features':[10,30,40,50,60]},
             {"bootstrap":[False], 'n_estimators':[3,10], 'max_features':[5,20, 50,80]}]

if __name__ == '__main__':
    data = Data(dataset_features, dataset_target,cat_features, num_features, "salary")
    models = Models(data)
    models.add_model(lin_alg)
    models.add_model(ridge_reg)
    models.add_model(rfr)
    models.add_model(gbr)
    models.run()


TypeError: unsupported operand type(s) for +: 'LinearRegression' and 'str'

In [12]:
for model in models.models:
    print(mean_squared_error(models.ytest,model.predict(models.Xtest)))
    print(models.model_info[model])

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

(5, 10)